Requirements:
- Have Milvus running locally on port 19530
- Libraries: pymilvus, numpy

In [1]:
import time
import numpy as np
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

In [2]:
fmt = "\n=== {:30} ===\n"
search_latency_fmt = "search latency = {:.4f}s"

print(fmt.format("start connecting to Milvus"))
connections.connect("default", host="localhost", port="19530")


=== start connecting to Milvus     ===



In [23]:
# check if collection exists
collection_name = "scouting"
hasScoutingCollection = utility.has_collection(collection_name)
print(f"Does collection scouting exist in Milvus: {hasScoutingCollection}")

Does collection scouting exist in Milvus: False


In [35]:
# create collection
dim=8
fields = [
    FieldSchema(name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=100), #pk = primary key
    FieldSchema(name="report_text", dtype=DataType.VARCHAR, max_length=1000),
    FieldSchema(name="report_length", dtype=DataType.INT16),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=dim)
]

schema = CollectionSchema(fields, "Scouting report vectors")
print(fmt.format("Creating collection 'scouting'"))
scouting_collection = Collection(collection_name, schema, consistency_level="Strong")


=== Creating collection 'scouting' ===



In [36]:
#insert new reports
rng = np.random.default_rng(seed=19530)
reports = [
    ["1","2", "3"],
    ["First report", "second report", "third report"],
    [1, 50, 1337],
    rng.random((3, dim)) # create random vectors for now for embedding
]

insert_result = scouting_collection.insert(reports)
scouting_collection.flush()
print(f"Number of reports in Milvus: {scouting_collection.num_entities}")

Number of reports in Milvus: 3


In [37]:
# create index
print(fmt.format("Start Creating index IVF_FLAT"))
index = {
    "index_type": "IVF_FLAT", # we cluster our data and only compare our query to the elements of the nearest cluster center https://milvus.io/docs/index.md#IVFFLAT
    "metric_type": "L2", #euclidean distance, could also use cosine here https://milvus.io/docs/metric.md
    "params": {"nlist": 128}, #nlist -> number of clusters
}

scouting_collection.create_index("embeddings", index)


=== Start Creating index IVF_FLAT  ===



Status(code=0, message=)

In [39]:
#need to load data into memory before searching
print(fmt.format("Start loading"))
scouting_collection.load() 


=== Start loading                  ===



In [28]:
# search based on vector similarity
print(fmt.format("Search on vector similarity"))
vectors_to_search = reports[-1][0:1]
print(vectors_to_search)
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10}, #number of clusters to search
}

start_time = time.time()
result = scouting_collection.search(vectors_to_search, "embeddings", search_params, limit=2, output_fields=["pk", "report_length","report_text"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, scouting report text field: {hit.entity.get('report_text')}")
print(search_latency_fmt.format(end_time - start_time))


=== Search on vector similarity    ===

[[0.6378742  0.43925104 0.13211584 0.46866668 0.74429647 0.03190612
  0.31691246 0.60253741]]
hit: id: 1, distance: 0.0, entity: {'report_text': 'First report', 'pk': '1', 'report_length': 1}, scouting report text field: First report
hit: id: 2, distance: 0.5949171185493469, entity: {'report_text': 'second report', 'pk': '2', 'report_length': 50}, scouting report text field: second report
search latency = 0.2032s


In [30]:
# search based on scalar filtering (ie our meta data like names, ids...)
print(fmt.format("Start querying with `report_length > 30`"))
start_time = time.time()
result = scouting_collection.query(expr="report_length > 30", output_fields=["pk", "report_text", "embeddings"])
end_time = time.time()

print(f"query result:\n-{result}")
print(search_latency_fmt.format(end_time - start_time))


=== Start querying with `report_length > 30` ===

query result:
-data: ["{'pk': '2', 'report_text': 'second report', 'embeddings': [0.9007387, 0.44944635, 0.18477614, 0.42930314, 0.40345728, 0.3957196, 0.6963897, 0.24356908]}", "{'pk': '3', 'report_text': 'third report', 'embeddings': [0.42512414, 0.5724385, 0.42719918, 0.8820724, 0.84478086, 0.6917027, 0.27135953, 0.9762772]}"] , extra_info: {'cost': 0}
search latency = 0.3621s


In [31]:
# hybrid search
print(fmt.format("Start hybrid searching with `report_length > 30`"))

start_time = time.time()
result = scouting_collection.search(vectors_to_search, "embeddings", search_params, limit=2, expr="report_length > 30", output_fields=["pk", "report_text"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, report_text field: {hit.entity.get('report_text')}")
print(search_latency_fmt.format(end_time - start_time))


=== Start hybrid searching with `report_length > 30` ===

hit: id: 2, distance: 0.5949171185493469, entity: {'pk': '2', 'report_text': 'second report'}, report_text field: second report
hit: id: 3, distance: 0.9081650972366333, entity: {'pk': '3', 'report_text': 'third report'}, report_text field: third report
search latency = 0.2280s


In [42]:
# delete entities
ids = [1] # delete report with id 1
expression = f'pk in ["{ids[0]}"]'

result = scouting_collection.query(expr=expression, output_fields=["pk", "report_text"])
print(f"query before delete by expr=`{expression}` -> result: \n-{result}\n")

scouting_collection.delete(expression)

result = scouting_collection.query(expr=expression, output_fields=["pk", "report_text"])
print(f"query after delete by expr=`{expression}` -> result: {result}\n")

query before delete by expr=`pk in ["1"]` -> result: 
-data: ["{'pk': '1', 'report_text': 'First report'}"] , extra_info: {'cost': 0}

query after delete by expr=`pk in ["1"]` -> result: data: [] , extra_info: {'cost': 0}



In [43]:
# drop collection
print(fmt.format("Drop collection"))
utility.drop_collection(collection_name)


=== Drop collection                ===

